The function computes the eigenvalues and eigenvectors of a three-dimensional symmetric matrix.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| C | array (3,3) | $ C $|
| [eig1, eig2, eig3] | list | list of eigenvalues (in increasing order) of $ C $|
| eigenvector | array (3,3) | eigenvectors of $ C $|

In [22]:
# Import numpy
import numpy as np

# Import math tools
from math import sqrt, acos, pi, cos

In [23]:
def eigen(C):
    '''The function computes the eigenvalues and eigenvectors of a three-dimensional symmetric matrix.
    
    Parameters:
        C: array(3,3), input matrix 
        
        
    Returns:
        [eig1, eig2, eig3]: list, eigenvalues of C in increasing order
        eigenvector: array(3,3): eigenvectors of C, stored column-wise. eigenvector1 = eigenvector[:,0]
    '''
    p1 = C[0,1]*C[1,0] + C[0,2]*C[2,0] + C[1,2]*C[2,1]
    
    if p1 == 0:
        # A is diagonal.
        eig1 = C[0,0]
        eig2 = C[1,1]
        eig3 = C[2,2]
    else:
        traceC = C[0,0]+C[1,1]+C[2,2]
        q = traceC/3               # trace(A) is the sum of all diagonal values
        p2 = (C[0, 0] - q)**2 + (C[1,1] - q)**2 + (C[2,2] - q)**2 + 2 * p1
        p = sqrt(p2 / 6)
        B = (1 / p) * (C - q * np.eye(3, 3))
        r = np.linalg.det(B) / 2

       # In exact arithmetic for a symmetric matrix  -1 <= r <= 1
       # but computation error can leave it slightly outside this range.
        if r <= -1:
            phi = pi / 3
        elif r >= 1:
            phi = 0
        else:
            phi = acos(r) / 3

       # the eigenvalues satisfy eig3 <= eig2 <= eig1
        eig1 = q + 2 * p * cos(phi)
        eig3 = q + 2 * p * cos(phi + (2*pi/3))
        eig2 = 3 * q - eig1 - eig3     # since trace(A) = eig1 + eig2 + eig3
    
    eigenvector = []
    
    for eig in [eig1, eig2, eig3]:
        if eig != C[0,0]:
            A = C[0,1]*C[1,0]/(eig-C[0,0])+C[1,1]-eig
            B = C[1,0]*C[0,2]/(eig-C[0,0])+C[1,2]
            
            # numerical threshold
            if np.abs(A) >= 10**(-8):
                s3 = 1
                s2 = -B/A
                s1 = C[0,1]/(eig-C[0,0]) * s2 + C[0,2]/(eig-C[0,0])
            else:
                s2 = 1
                s3 = 0
                s1 = C[0,1]/(eig-C[0,0])
        
        # Assume that C[0,0] is equal to the eigenvalue but C[1,1] is not
        if eig == C[0, 0] and eig != C[1, 1]:
        
            A = C[1, 2] * C[2, 1]/(eig-C[1, 1])+C[2, 2]-eig
            B = C[1, 2] * C[1, 0]/(eig-C[1, 1])+C[2, 0]
            if A != 0:
                s1 = 1
                s3 = -B/A
                s2 = C[1, 0]/(eig-C[1, 1])+C[1, 2]/(eig-C[1, 1])*s3
            elif A== 0 and B!= 0:
                s3 = 1
                s1 = 0
                s2 = C[1, 2]/(eig-C[1,1])
        
        # Assume that C[0,0] and C[1,1] are equal to the eigenvalue but C[2,2] is not
        if C[0, 0] == eig and C[1, 1] == eig and C[2, 2] != eig:
            A = C[1, 2] * C[2, 0]/(eig-C[2, 2])+C[1, 0]
            B = C[2, 1]*C[1,2]/(eig-C[2, 2])
            if A!=0:
                s2 =1
                s1 = -B/A
                s3 = C[2,0]/(eig-C[2,2])*s1+C[2,1]/(eig-C[2,2])*s2
            elif A == 0 and B!= 0:
                s1 = 1
                s2 = 0
                s3 = C[2,0]/(eig-C[2,2])
                
        # Assume that Z is the eigendirection (trivial 2Dflow)
        if C[0, 0] == eig and C[1, 1] == eig and C[2, 2] == eig and eig == 0:
            s1 = 0
            s2 = 0
            s3 = 1
    
        # normalize eigenvector
        norm = sqrt(s1**2+s2**2+s3**2)
        s1 = s1/norm
        s2 = s2/norm
        s3 = s3/norm

        eigenvector.append([s1, s2, s3])
    
    eig = [eig1,eig2,eig3]
    
    eig_sorted = [x for _, x in sorted(zip(eig, eig))]
    eigenvector_sorted = [x for _, x in sorted(zip(eig, eigenvector))]
    
    return np.array(eig_sorted), np.array(eigenvector_sorted).transpose()